# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 22 2022 8:30AM,253612,10,MSP216978,"Methapharm, Inc.",Released
1,Dec 22 2022 8:30AM,253612,10,MSP216982,"Methapharm, Inc.",Released
2,Dec 22 2022 8:30AM,253615,10,MSP216979,"Methapharm, Inc.",Released
3,Dec 22 2022 8:30AM,253615,10,MSP216980,"Methapharm, Inc.",Released
4,Dec 22 2022 8:30AM,253615,10,MSP216981,"Methapharm, Inc.",Released
5,Dec 22 2022 8:30AM,253636,10,Eme-106767,Emerginnova,Released
6,Dec 22 2022 8:30AM,253637,10,0086263789,ISDIN Corporation,Released
7,Dec 22 2022 8:30AM,253638,10,0086263498,ISDIN Corporation,Released
8,Dec 22 2022 8:30AM,253638,10,0086263501,ISDIN Corporation,Released
9,Dec 22 2022 8:30AM,253638,10,0086263773,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,253638,Released,10
44,253639,Released,7
45,253640,Released,1
46,253642,Released,5
47,253643,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253638,NaN,NaN,10.0
253639,NaN,NaN,7.0
253640,NaN,NaN,1.0
253642,NaN,NaN,5.0
253643,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253560,11.0,12.0,65.0
253563,2.0,5.0,2.0
253564,0.0,0.0,1.0
253570,0.0,0.0,23.0
253572,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253560,11,12,65
253563,2,5,2
253564,0,0,1
253570,0,0,23
253572,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253560,11,12,65
1,253563,2,5,2
2,253564,0,0,1
3,253570,0,0,23
4,253572,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253560,11,12,65
1,253563,2,5,2
2,253564,,,1
3,253570,,,23
4,253572,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc."
2,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc."
5,Dec 22 2022 8:30AM,253636,10,Emerginnova
6,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation
7,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation
17,Dec 22 2022 8:30AM,253639,10,ISDIN Corporation
24,Dec 22 2022 8:30AM,253640,10,Methapharm-G
25,Dec 22 2022 8:30AM,253642,10,"Methapharm, Inc."
29,Dec 22 2022 8:30AM,253642,10,Methapharm-G
30,Dec 22 2022 8:30AM,253643,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc.",,,2
1,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc.",,,3
2,Dec 22 2022 8:30AM,253636,10,Emerginnova,,,1
3,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation,,,1
4,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation,,,10
5,Dec 22 2022 8:30AM,253639,10,ISDIN Corporation,,,7
6,Dec 22 2022 8:30AM,253640,10,Methapharm-G,,,1
7,Dec 22 2022 8:30AM,253642,10,"Methapharm, Inc.",,,5
8,Dec 22 2022 8:30AM,253642,10,Methapharm-G,,,5
9,Dec 22 2022 8:30AM,253643,10,"Methapharm, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc.",2,,
1,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc.",3,,
2,Dec 22 2022 8:30AM,253636,10,Emerginnova,1,,
3,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation,1,,
4,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation,10,,
5,Dec 22 2022 8:30AM,253639,10,ISDIN Corporation,7,,
6,Dec 22 2022 8:30AM,253640,10,Methapharm-G,1,,
7,Dec 22 2022 8:30AM,253642,10,"Methapharm, Inc.",5,,
8,Dec 22 2022 8:30AM,253642,10,Methapharm-G,5,,
9,Dec 22 2022 8:30AM,253643,10,"Methapharm, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc.",2,,
1,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc.",3,,
2,Dec 22 2022 8:30AM,253636,10,Emerginnova,1,,
3,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation,1,,
4,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc.",2,NaN,NaN
1,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc.",3,NaN,NaN
2,Dec 22 2022 8:30AM,253636,10,Emerginnova,1,NaN,NaN
3,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation,1,NaN,NaN
4,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation,10,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 8:30AM,253612,10,"Methapharm, Inc.",2,0.0,0.0
1,Dec 22 2022 8:30AM,253615,10,"Methapharm, Inc.",3,0.0,0.0
2,Dec 22 2022 8:30AM,253636,10,Emerginnova,1,0.0,0.0
3,Dec 22 2022 8:30AM,253637,10,ISDIN Corporation,1,0.0,0.0
4,Dec 22 2022 8:30AM,253638,10,ISDIN Corporation,10,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7101163,97,15.0,2.0
16,253578,1,0.0,0.0
17,760877,3,0.0,0.0
19,1775158,73,13.0,11.0
20,507176,2,1.0,0.0
21,253603,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7101163,97,15.0,2.0
1,16,253578,1,0.0,0.0
2,17,760877,3,0.0,0.0
3,19,1775158,73,13.0,11.0
4,20,507176,2,1.0,0.0
5,21,253603,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,97,15.0,2.0
1,16,1,0.0,0.0
2,17,3,0.0,0.0
3,19,73,13.0,11.0
4,20,2,1.0,0.0
5,21,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,97.0
1,16,Released,1.0
2,17,Released,3.0
3,19,Released,73.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,17,19,20,21
Status,,,,,,
Completed,2.0,0.0,0.0,11.0,0.0,0.0
Executing,15.0,0.0,0.0,13.0,1.0,0.0
Released,97.0,1.0,3.0,73.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,17,19,20,21
0,Completed,2.0,0.0,0.0,11.0,0.0,0.0
1,Executing,15.0,0.0,0.0,13.0,1.0,0.0
2,Released,97.0,1.0,3.0,73.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,17,19,20,21
0,Completed,2.0,0.0,0.0,11.0,0.0,0.0
1,Executing,15.0,0.0,0.0,13.0,1.0,0.0
2,Released,97.0,1.0,3.0,73.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()